In [15]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [19]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [20]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge = np.maximum(1 - y * (X @ w), 0)
    L2 = (lambda_ / 2) * (w.T @ w)
    return np.sum(hinge) + L2

calculate_primal_objective(np.array([-1, 1, 1, -1]), np.array([[1,6,3],[3,1,9],[9,5,2], [2,1,12]]), np.array([2,1,4]), 0.1)

76.05

In [21]:
from helpers import predict_labels
from refik_lib.scores import compute_accuracy

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    pred_y = predict_labels(w, X)
    return compute_accuracy(pred_y, y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [89]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    sub_g = 1 if 1 - y_n * (x_n @ w) > 0 else 0
    g = -(y_n * x_n) * sub_g
    return num_examples * g + (lambda_ * w)

The reason we are muliplying with num examples is because our loss function is a sum and if we don't the expected value of our stocastic sampling falls short of estimating the true gradient.

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [90]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=95371402772033.97
iteration=10000, cost=2443529991186.499
iteration=20000, cost=1945085552239.7935
iteration=30000, cost=1668282349121.041
iteration=40000, cost=1478299257678.1948
iteration=50000, cost=1340672368702.6096
iteration=60000, cost=1224710447659.6316
iteration=70000, cost=1130870638661.1887
iteration=80000, cost=1051743720341.573
iteration=90000, cost=980352065811.9935
training accuracy = 0.623


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [104]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    # directly from the solutions to theory questions sheet
    gamma = (lambda_ / (x_n.T @ x_n)) * (1 - y_n * w.T @ x_n)
    unbounded_alpha = old_alpha_n + gamma
    alpha = np.clip(unbounded_alpha, 0, 1)
    return w, alpha

In [105]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    Y = np.diag(y)
    return np.sum(alpha) - (1 / 2*lambda_) * alpha.T * Y @ X @ X.T @ Y * alpha

In [106]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

TypeError: only size-1 arrays can be converted to Python scalars